# k-Means
Der k-Means-Algorithmus ist eines der beliebtesten Verfahren zur Clusteranalyse. Es lässt sich sehr einfach implementieren und findet schnell entsprechende Clusterzentren.

## Inhaltsverzeichnis
- [Der Algorithmus](#Der-Algorithmus)
- [Visualisierung](#Visualisierung)
- [Erweiterte Implementierungen](#Erweiterte-Implementierungen)
- [Probleme](#Probleme)

## Der Algorithmus
k-Means funktioniert nach folgendem Verfahren:

1. Wähle $k$ Clusterzentren zufällig aus dem Datensatz. Die Anzahl $k$ muss also vorher bekannt sein.
2. wiederhole bis zur Konvergenz:
  1. Jeder Punkt wird dem Cluster zugeordnet, dessen Zentrum er am nächsten liegt.
  2. Jedes Clusterzentrum wird als Mittelpunkt aller zugehörigen Punkte neu bestimmt.

Die folgende Funktion arbeitet als Generator und gibt immer wieder neue Clusterzentren zurück, bis sie nicht mehr aufgerufen wird. Die Prüfung des Abbruchkriteriums fällt damit in die Verantwortung des aufrufenden Programmbestandteils.

In [ ]:
import numpy as np
import random

def k_means(xs, ys, k):
    # Die Clusterzentren werden zufällig gewählt.
    idx = np.random.choice(np.arange(len(xs)), k, replace=False)

    centroids_x = xs[idx]
    centroids_y = ys[idx]

    yield centroids_x, centroids_y

    # Die Clusterzentren werden wiederholt neu bestimmt.
    while True:
        # Die Variablen dienen dem Speichern der neuen
        # Position der Clusterzentren.
        new_centroids_x = np.zeros(k)
        new_centroids_y = np.zeros(k)

        centroids_len = np.zeros(k)

        # Die Clusterzugehörigkeit wird für alle Punkte
        # bestimmt.
        for x, y in zip(xs, ys):
            # Dazu wird die Distanz zu jedem Cluster-
            # mittelpunkt berechnet. Das Minimum
            # beschreibt den nahegelegendsten Cluster.
            distances = np.sqrt((centroids_x - x) ** 2 + (centroids_y - y) ** 2)
            centroid_index = distances.argmin()

            # Die Koordinaten des Punktes werden zur
            # Neubestimmung des Clusterzentrums
            # gespeichert.
            new_centroids_x[centroid_index] += x
            new_centroids_y[centroid_index] += y

            centroids_len[centroid_index] += 1

        # Clustermittelpunkte, denen keine Punkte zu-
        # geordnet wurden, werden erneut zufällig
        # verteilt.
        for i in range(len(centroids_len)):
            if centroids_len[i] == 0:
                new_centroids_x[i] = np.random.uniform(min_x, max_x)
                new_centroids_y[i] = np.random.uniform(min_y, max_y)
                centroids_len[i] = 1

        # Die summierten Punkte werden durch die Anzahl
        # geteilt, um neue Clustermittelpunkte zu
        # erhalten.
        centroids_x = new_centroids_x / centroids_len
        centroids_y = new_centroids_y / centroids_len

        yield centroids_x, centroids_y

## Visualisierung

In [ ]:
import pandas as pd
import plotly.express as px

from animation import create_animation

# Datensatz laden und zur Animation vorbereiten
df = pd.read_csv('clustering_example1.csv', dtype={'c': str})
df_anim = create_animation(df, k_means, 15)

# Animation anzeigen
px.scatter(df_anim, x='x', y='y',
           animation_frame='frame', animation_group='group',
           color='class', symbol='type', size='type',
           color_discrete_sequence=px.colors.qualitative.Light24)

## Erweiterte Implementierungen
Wie Sie gesehen haben, hängt der Algorithmus von der Initialisierung des Zufallszahlengenerators ab. Es kann sich daher lohnen, die Methode mehrfach auszuführen und das beste Ergebnis auszuwählen. Eine fertige Implementierung, die den k-Means-Algorithmus mehrfach ausführt und weitere Verbesserungen enthält, entstammt dem Paket `sklearn`.

Initialisieren Sie dazu ein Objekt der Klasse `KMeans` und übergeben Sie dem Parameter die Anzahl der Clusterzentren. Rufen Sie anschließend `fit_predict` mit einem DataFrame auf, das die Angaben zu den entsprechenden Dimensionen enthält. Zurückgegeben wird ein Array aus Klassenindizes.

In [ ]:
from sklearn.cluster import KMeans

df_kmeans = df.copy()[['x', 'y']]
df_kmeans['c'] = KMeans(15).fit_predict(df).astype(str)

px.scatter(df_kmeans, x='x', y='y', color='c',
           color_discrete_sequence=px.colors.qualitative.Light24)

## Probleme
Der k-Means Algorithmus eignet sich jedoch nicht für alle Daten. Mengen, die nicht konvex sind, können beispielsweise nicht korrekt in Cluster eingeteilt werden.

In [ ]:
# Datensatz laden
df = pd.read_csv('clustering_example2.csv', dtype={'c': str})

# DataFrame zur Animation vorbereiten
df_anim = create_animation(df, k_means, 3)

# Animation anzeigen
px.scatter(df_anim, x='x', y='y',
           animation_frame='frame', animation_group='group',
           color='class', symbol='type', size='type',
           color_discrete_sequence=px.colors.qualitative.Light24)